# Chronic Disease Collection

Chronic Disease Collection


In [1]:
#import ambry.jupyter
import pandas as pd
from ambry import get_library
l = get_library()
b = l.bundle('ucla.edu-askchisne-hdp-0.0.1')
w = b.warehouse('working')
w.clean()
w.dsn

'sqlite://///Users/eric/proj/virt/ambry-develop/data/build/ucla.edu/askchisne-hdp/warehouses/working.db'

In [2]:
## %%warehouse_query w
w.query("""
-- Get only counties in California
CREATE TABLE geo AS 
SELECT gvid, name AS county_name  --, geometry 
FROM census.gov-tiger-2015-counties
WHERE statefp = 6;

-- Not including the 90% margin column because for California counties, the population estimates are 
-- controlled ( fixed by the Population Estimates program ) and therefore have no margin. 
--- NOTE! This is the 2014 1 year population estimate
CREATE TABLE population AS 
SELECT b01003.gvid, b01003001 AS total_pop,  b09001001 AS u18_pop ,  b09001001_m90 AS u18_pop_m90,
(b01003001-b09001001) AS over18_pop, b09001001_m90 AS over18_pop_m90
FROM census.gov-acs-p1ye2014-b01003 AS b01003
LEFT JOIN census.gov-acs-p1ye2014-b09001 AS b09001 on b09001.gvid = b01003.gvid
WHERE b01003.sumlevel = 50 AND b01003.stusab = 'ca';

INDEX geo(gvid);

INDEX population(gvid);
""").close()


Installing 'p0520tS002'
Installing 'p0520iS002'
Installing 'p04M02C002'


In [3]:
## %%warehouse_query w
w.query("""
CREATE VIEW chronic_index AS
SELECT geo.gvid, geo.county_name, population.pop
FROM geo
LEFT JOIN population ON population.gvid = geo.gvid
""").close()


In [4]:
from geoid.civick import State, County
str(County(6,0)) # The code for the whole state

'0O0600'

In [5]:
# A Single record for the summary of the whole state. 
## %%warehouse_query w
w.query("""
CREATE VIEW state_chronic AS
SELECT '0O0600' as gvid, 'CA' AS state, 'statewide' AS county, asthmaa.population AS over_18_pop,
(asthmaa.estimate*100) AS asthmaa_pct, (asthmaa.se*100) AS asthmaa_pct_se, (asthmaa.estimate * asthmaa.population) AS asthmaa_count,
(diaba.estimate*100) AS diaba_pct, (diaba.se*100) AS diaba_pct_se, (diaba.estimate * diaba.population) AS diaba_count,
(heartda.estimate*100) AS heartda_pct, (heartda.se*100) AS heartda_pct_se, (heartda.estimate * heartda.population) AS heartda_count,
(psychdist.estimate*100) AS psychdist_pct, (psychdist.se*100) AS psychdist_pct_se, (psychdist.estimate * psychdist.population) AS psychdist_count
FROM ucla.edu-askchisne-asthmaa-state AS asthmaa 
LEFT JOIN ucla.edu-askchisne-diaba-state   AS diaba  
LEFT JOIN ucla.edu-askchisne-heartda-state AS heartda       
LEFT JOIN ucla.edu-askchisne-dstrsa-state  AS psychdist;
""").close()

Installing 'p04Q076002'
Installing 'p04Q06W002'
Installing 'p04Q09k002'
Installing 'p04Q08J002'


In [6]:
w.dataframe("SELECT * FROM state_chronic").head()

,gvid,state,county,over_18_pop,asthmaa_pct,asthmaa_pct_se,asthmaa_count,diaba_pct,diaba_pct_se,diaba_count,heartda_pct,heartda_pct_se,heartda_count,psychdist_pct,psychdist_pct_se,psychdist_count
0,0O0600,CA,statewide,27796500,13.7,0.281,3808120.5,8.4,0.211,2334906,6.3,0.173,1751179.5,7.9,0.232,2195923.5


In [7]:
## %%warehouse_query w
w.query("""
CREATE VIEW county_chronic AS
SELECT geo.gvid, 'CA' as state, geo.county_name as county, asthmaa.population AS over_18_pop,
(asthmaa.estimate*100) AS asthmaa_pct, (asthmaa.se*100) AS asthmaa_pct_se, (asthmaa.estimate * asthmaa.population) AS asthmaa_count,
(diaba.estimate*100) AS diaba_pct, (diaba.se*100) AS diaba_pct_se, (diaba.estimate * diaba.population) AS diaba_count,
(heartda.estimate*100) AS heartda_pct, (heartda.se*100) AS heartda_pct_se, (heartda.estimate * heartda.population) AS heartda_count,
(psychdist.estimate*100) AS psychdist_pct, (psychdist.se*100) AS psychdist_pct_se, (psychdist.estimate * psychdist.population) AS psychdist_count
FROM geo
LEFT JOIN population ON population.gvid = geo.gvid
LEFT JOIN ucla.edu-askchisne-asthmaa-counties AS asthmaa ON asthmaa.gvid = geo.gvid
LEFT JOIN ucla.edu-askchisne-diaba-counties   AS diaba   ON diaba.gvid = geo.gvid
LEFT JOIN ucla.edu-askchisne-heartda-counties AS heartda ON heartda.gvid = geo.gvid           
LEFT JOIN ucla.edu-askchisne-dstrsa-counties  AS psychdist ON psychdist.gvid = geo.gvid;

CREATE VIEW chronic AS
SELECT * FROM county_chronic
UNION
SELECT * FROM state_chronic
""").close()


Installing 'p04Q08o002'
Installing 'p04Q0aG002'
Installing 'p04Q088002'
Installing 'p04Q0av002'


In [8]:
df = w.dataframe('SELECT * FROM chronic').set_index('gvid')
df

,state,county,over_18_pop,asthmaa_pct,asthmaa_pct_se,asthmaa_count,diaba_pct,diaba_pct_se,diaba_count,heartda_pct,heartda_pct_se,heartda_count,psychdist_pct,psychdist_pct_se,psychdist_count
gvid,,,,,,,,,,,,,,,
0O0600,CA,statewide,27796500,13.7,0.281,3808120.5,8.4,0.211,2334906.0,6.3,0.173,1751179.5,7.9,0.232,2195923.5
0O0601,CA,Alameda,1158300,16.4,1.381,189961.2,7.0,0.773,81081.0,5.8,0.394,67181.4,7.7,0.745,89189.1
0O0603,CA,Alpine,800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0O0605,CA,Amador,27200,13.2,1.339,3590.4,7.9,0.543,2148.8,10.5,0.537,2856.0,9.6,1.228,2611.2
0O0607,CA,Butte,170200,18.3,1.685,31146.6,7.7,0.455,13105.4,8.2,0.449,13956.4,10.8,0.896,18381.6
0O0609,CA,Calaveras,36200,13.8,1.373,4995.6,8.1,0.584,2932.2,10.3,0.579,3728.6,10.2,1.308,3692.4
0O060B,CA,Los Angeles,7402100,12.2,0.490,903056.2,8.8,0.382,651384.8,5.9,0.221,436723.9,8.0,0.410,592168.0
0O060D,CA,Madera,99900,15.2,1.902,15184.8,11.4,0.981,11388.6,7.7,0.308,7692.3,10.4,1.296,10389.6
0O060F,CA,Marin,193900,12.2,1.107,23655.8,6.0,0.593,11634.0,8.1,0.379,15705.9,5.9,0.650,11440.1
